In [2]:
import numpy as np

## Single Neuron

## Example

In [4]:
class NeuronNode:
    def __init__(self, dim):
        self.dim = dim
        self.weights = [0] * dim
        self.bias = 0
    
    def trigger(self, inputs):
        if len(inputs) != len(self.weights): raise ValueError("Invalid Input length")
        return np.dot(self.weights, inputs) + self.bias # dot(Input, Weight) + bias

## Neural Network with a forward pass

This neural network is made of several units of NeuronNode

In [3]:
def relu(x):
    pass

In [5]:
class NeuralNetwork:
    def __init__(self, act_fun = relu):
        self.act_fun = act_fun
        self.layers = []
    
    def add_layer(self, neurons):
        self.layers.append(neurons)
    
    def forward(self, inputs):
        current_input = inputs
        current_level = 0
        curr_output = [] * len(self.layers[0])

        while current_level < len(self.layers):
            curr_layer = self.layers[current_level]
            if any(len(current_input) != neuron.dim for neuron in curr_layer):
                raise Exception('Dimension mismatch')
            curr_output = [neuron.trigger(current_input) for neuron in curr_layer]
            current_input = curr_output
            current_level += 1
        else:
            return curr_output

This neural network is made using weight matrices
1. A weight matrix exists for each layer
2. The dimension of the weight matrix is no of neurons * input dim

In [20]:
class NeuralNetwork:
    def __init__(self, act_fun = relu):
        self.act_fun = act_fun
        self.layers = []
    
    def add_layer(self, weight_mat, biases):
        if len(biases) != len(weight_mat) or any(len(weight_mat[i]) != len(weight_mat[0])
            for i in range(1, len(weight_mat))):
            raise Exception("Dimension Invalid")
        
        self.layers.append((weight_mat, biases))
    
    def forward(self, input):
        current_layer = 0
        current_input = input
        current_output = []

        while current_layer < len(self.layers):
            layer = self.layers[current_layer]  # current weight_matrix
            weights, biases = layer
            
            if len(current_input)!= biases or any(len(current_input) != len(weights[i])
                                                  for i in range(len(weights))):
                raise Exception("Invalid Dimension")
            
            current_output = np.dot(weights, current_input) + biases
            current_layer += 1
            current_input = current_output
        return current_output

In [16]:
nn = NeuralNetwork()

In [17]:
inputs = [1, 2, 3, 2.5]
weights = [
    [0.2, 0.8, -0.5, 1],
    [0.5, -0.91, 0.26, -0.5],
    [-0.26, -0.27, 0.17, 0.87]
]
biases = [2, 3, 0.5]

In [18]:
nn.add_layer(weights, biases)

In [19]:
nn.forward(inputs)

array([4.8  , 1.21 , 2.385])